In [1]:
import os
import pandas as pd
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

In [2]:
from util import *

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
class TestDataset(Dataset):
    def __init__(self, args, transform=None):
        self.test_dir = args['test_dir']
        self.test_csv_dir = args['test_csv_dir']
        self.test_csv_exist_dir = args['test_csv_exist_dir']
        self.args = args
        self.transform = transform
        self.test_image = list()
        self.test_label = list()
        if not os.path.isfile(self.test_csv_exist_dir):
            self.test_csv = pd.read_csv(self.test_csv_dir, encoding='utf-8')
            self.test_csv_exist = self.test_csv.copy()
            self.load_full_data()
            self.test_csv_exist.to_csv(self.test_csv_exist_dir, index=False)
        else :
            self.load_exist_data()

    def load_full_data(self):
        for i in tqdm(range(len(self.test_csv))):
            filename = self.test_csv['id'][i]
            fullpath = glob(self.test_dir + "*/" + filename.replace('[', '[[]') + ".JPG")[0]
            label = self.test_csv['id'][i]

            self.test_csv_exist.loc[i,'id'] = fullpath
            self.test_image.append(fullpath)
            self.test_label.append(label)


    def load_exist_data(self):
        self.test_csv_exist = pd.read_csv(self.test_csv_exist_dir)
        for i in tqdm(range(len(self.test_csv_exist))):
            fullpath = self.test_csv_exist['id'][i]
            label = self.test_csv_exist['id'][i]

            self.test_image.append(fullpath)
            self.test_label.append(label)


    def __len__(self):
        return len(self.test_image)

    def __getitem__(self, idx):
        image = Image.open(self.test_image[idx])
        if self.transform is not None:
            image = np.array(image)
            augmented = self.transform(image=image)
            image = augmented['image']
        else:
            image = image.resize((self.args['image_size'], self.args['image_size']))
            image = np.array(image) / 255.
            image = np.transpose(image, axes=(2, 0, 1))
        label = self.test_label[idx]
        return image, label

In [5]:
args = {'image_size': 224,
        'test_dir': './public/test',
        'test_csv_dir': './public/sample_submission.csv',
        'test_csv_exist_dir': './public/sample_submission_exist.csv'}

In [6]:
test_transform = A.Compose([
    A.SmallestMaxSize(224),
    A.Normalize(mean=[0.4452, 0.4457, 0.4464],
                std=[0.2592, 0.2596, 0.2600]),
    ToTensorV2(),
])

test_dataset = TestDataset(args, transform=test_transform)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset,
                         sampler=test_sampler,
                         batch_size=20,
                         shuffle=False,
                         num_workers=10,
                         pin_memory=False,
                         drop_last=False)

100%|██████████| 37964/37964 [00:03<00:00, 10104.69it/s]


In [7]:
def eval_best(model, model_dir, target, test_loader, depth=4, n_classes=1049, feat_dim=128, batch_size=10,
              test_csv_dir='./public/sample_submission.csv',
              test_csv_submission_dir='./public/my_submission.csv'):
    model.load_state_dict(torch.load(os.path.join(model_dir, target)))
    model.eval()
    submission = pd.read_csv(test_csv_dir)
    result = []
    for iter, (image, label) in enumerate(tqdm(test_loader)):
        image = image.cuda()
        pred = model(image)
        pred = nn.Softmax(dim=1)(pred)
        pred = pred.detach().cpu().numpy()
        landmark_ids = np.argmax(pred, axis=1)
        for offset, landmark_id in enumerate(landmark_ids):
            confidence = pred[offset, landmark_id]
            cur_idx = (iter*batch_size) + offset
            submission.loc[cur_idx, 'landmark_id'] = landmark_id
            submission.loc[cur_idx, 'conf'] = confidence
        result.append(pred)
    submission.to_csv(test_csv_submission_dir, index=False)
    print(f'Save submission: {len(submission)}')
    result = np.concatenate(result)
    return result

In [10]:
save_list = [
    'save/efficient4_224_0',
    'save/efficient4_224_1',
    'save/efficient4_224_2',
    'save/efficient4_224_3',
    'save/ef4_224_2_68_bak',
    'save/ef4_224_3_68_bak'
]
target_list = [
    'epoch_067.pth',
    'epoch_067.pth',
    'epoch_067.pth',
    'epoch_067.pth',
    'epoch_053.pth',
    'epoch_052.pth',
]

In [11]:
result = []
for i, (model_dir, target) in enumerate(tqdm(zip(save_list, target_list))):
    model = EfficientNetEncoderHead(depth=4, num_classes=1049, feat_dim=128)
    model.cuda()
    ret = eval_best(model, model_dir, target, test_loader, depth=4, n_classes=1049, feat_dim=128, batch_size=20,
                    test_csv_dir='./public/sample_submission.csv',
                    test_csv_submission_dir=f'./public/submission_last_{i}.csv')
    result.append(ret)

0it [00:00, ?it/s]

Loaded pretrained weights for efficientnet-b4



 16%|█▌        | 599/3797 [00:58<05:13, 10.21it/s]


 31%|███▏      | 1193/3797 [01:55<04:12, 10.33it/s]


 47%|████▋     | 1776/3797 [02:52<03:17, 10.24it/s]


...

 31%|███       | 1185/3797 [01:55<04:19, 10.07it/s]


 38%|███▊      | 1455/3797 [02:21<03:45, 10.37it/s]

 77%|███████▋  | 2910/3797 [04:41<01:24, 10.47it/s]


 92%|█████████▏| 3493/3797 [05:38<00:29, 10.27it/s]


100%|██████████| 3797/3797 [06:07<00:00, 10.32it/s]
6it [37:08, 371.37s/it]

Save submission: 37964


In [12]:
pred = np.mean(result, axis=0)

In [13]:
pred.shape

(37964, 1049)

In [14]:
landmark_ids = np.argmax(pred, axis=1)

In [15]:
confidence = []
for i, cls in enumerate(landmark_ids):
    confidence.append(pred[i, cls])        

In [16]:
confidence = np.hstack(confidence)
confidence.shape

(37964,)

In [17]:
submission = pd.read_csv('./public/sample_submission.csv')

In [18]:
submission.loc[:, 'landmark_id'] = landmark_ids
submission.loc[:, 'conf'] = confidence

In [ ]:
# submission.to_csv('submission/ensemble_final.csv', index=False)
submission.to_csv('submission/ensemble_acc.csv', index=False)